# Imports 

In [1]:
from collections import Counter
import pandas as pd
import nltk
import string
from nltk.tokenize import word_tokenize

import math

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

# Load the data

In [2]:
df = pd.read_csv(r'../input/data-science-data-analyst-and-ml-job-indeed/job_dataset.csv')

In [3]:
df.head()

job_title          company  \
0  Technology Lead : Data Science I Machine Learning  Infosys Limited   
1                Software Engineer, Machine Learning           Google   
2        Experienced Over The Shoulder Mechanic- C17           BOEING   
3                                     Data Scientist  Infosys Limited   
4                                   Python Developer  Infosys Limited   

                             job_location  \
0                    Bengaluru, Karnataka   
1                    Bengaluru, Karnataka   
2  Remote in Air Force Station, Karnataka   
3                    Bengaluru, Karnataka   
4                    Bengaluru, Karnataka   

                                         job_summary  \
0  A day in the life of an Infoscion • As part of...   
1  2 years of relevant work experience in machine...   
2  This position will focus on supporting the Boe...   
3  Anchor ML development track in a client projec...   
4  Should have a decent understanding of the Mach...   

                   post_date       today job_salary  \
0  PostedPosted 30+ days ago  2022-07-26        NaN   
1  PostedPosted 30+ days ago  2022-07-26        NaN   
2    PostedPosted 3 days ago  2022-07-26        NaN   
3  PostedPosted 30+ days ago  2022-07-26        NaN   
4  PostedPosted 30+ days ago  2022-07-26        NaN   

                                             job_url  
0  https://in.indeed.com/pagead/clk?mo=r&ad=-6NYl...  
1  https://in.indeed.com/pagead/clk?mo=r&ad=-6NYl...  
2  https://in.indeed.com/pagead/clk?mo=r&ad=-6NYl...  
3  https://in.indeed.com/pagead/clk?mo=r&ad=-6NYl...  
4  https://in.indeed.com/pagead/clk?mo=r&ad=-6NYl...

In [4]:
df.shape

(1583, 8)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1583 entries, 0 to 1582
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   job_title     1583 non-null   object
 1   company       1583 non-null   object
 2   job_location  1583 non-null   object
 3   job_summary   1583 non-null   object
 4   post_date     1583 non-null   object
 5   today         1583 non-null   object
 6   job_salary    405 non-null    object
 7   job_url       1583 non-null   object
dtypes: object(8)
memory usage: 99.1+ KB


In [6]:
df.columns

Index(['job_title', 'company', 'job_location', 'job_summary', 'post_date',
       'today', 'job_salary', 'job_url'],
      dtype='object')

# Forming the Lists of Keywords

In [7]:
# got these keywords by looking at some examples and using existing knowledge.
tool_keywords1 = ['python', 'pytorch', 'sql', 'mxnet', 'mlflow', 'einstein', 'theano', 'pyspark', 'solr', 'mahout', 
 'cassandra', 'aws', 'powerpoint', 'spark', 'pig', 'sas', 'java', 'nosql', 'docker', 'salesforce', 'scala', 'r',
 'c', 'c++', 'net', 'tableau', 'pandas', 'scikitlearn', 'sklearn', 'matlab', 'scala', 'keras', 'tensorflow', 'clojure',
 'caffe', 'scipy', 'numpy', 'matplotlib', 'vba', 'spss', 'linux', 'azure', 'cloud', 'gcp', 'mongodb', 'mysql', 'oracle', 
 'redshift', 'snowflake', 'kafka', 'javascript', 'qlik', 'jupyter', 'perl', 'bigquery', 'unix', 'react',
 'scikit', 'powerbi', 's3', 'ec2', 'lambda', 'ssrs', 'kubernetes', 'hana', 'spacy', 'tf', 'django', 'sagemaker',
 'seaborn', 'mllib', 'github', 'git', 'elasticsearch', 'splunk', 'airflow', 'looker', 'rapidminer', 'birt', 'pentaho', 
 'jquery', 'nodejs', 'd3', 'plotly', 'bokeh', 'xgboost', 'rstudio', 'shiny', 'dash', 'h20', 'h2o', 'hadoop', 'mapreduce', 
 'hive', 'cognos', 'angular', 'nltk', 'flask', 'node', 'firebase', 'bigtable', 'rust', 'php', 'cntk', 'lightgbm', 
 'kubeflow', 'rpython', 'unixlinux', 'postgressql', 'postgresql', 'postgres', 'hbase', 'dask', 'ruby', 'julia', 'tensor',
# added r packages doesn't seem to impact the result
 'dplyr','ggplot2','esquisse','bioconductor','shiny','lubridate','knitr','mlr','quanteda','dt','rcrawler','caret','rmarkdown',
 'leaflet','janitor','ggvis','plotly','rcharts','rbokeh','broom','stringr','magrittr','slidify','rvest',
 'rmysql','rsqlite','prophet','glmnet','text2vec','snowballc','quantmod','rstan','swirl','datasciencer']


# another set of keywords that are longer than one word.
tool_keywords2 = set(['amazon web services', 'google cloud', 'sql server'])

In [8]:
# hard skills/knowledge required.
skill_keywords1 = set(['statistics', 'cleansing', 'chatbot', 'cleaning', 'blockchain', 'causality', 'correlation', 'bandit', 'anomaly', 'kpi',
 'dashboard', 'geospatial', 'ocr', 'econometrics', 'pca', 'gis', 'svm', 'svd', 'tuning', 'hyperparameter', 'hypothesis',
 'salesforcecom', 'segmentation', 'biostatistics', 'unsupervised', 'supervised', 'exploratory',
 'recommender', 'recommendations', 'research', 'sequencing', 'probability', 'reinforcement', 'graph', 'bioinformatics',
 'chi', 'knn', 'outlier', 'etl', 'normalization', 'classification', 'optimizing', 'prediction', 'forecasting',
 'clustering', 'cluster', 'optimization', 'visualization', 'nlp', 'c#',
 'regression', 'logistic', 'nn', 'cnn', 'glm',
 'rnn', 'lstm', 'gbm', 'boosting', 'recurrent', 'convolutional', 'bayesian',
 'bayes'])


# another set of keywords that are longer than one word.
skill_keywords2 = set(['random forest', 'natural language processing', 'machine learning', 'decision tree', 'deep learning', 'experimental design',
 'time series', 'nearest neighbors', 'neural network', 'support vector machine', 'computer vision', 'machine vision', 'dimensionality reduction', 
 'text analytics', 'power bi', 'a/b testing', 'ab testing', 'chat bot', 'data mining'])

In [9]:
degree_dict = {'bs': 1, 'bachelor': 1, 'undergraduate': 1, 
               'master': 2, 'graduate': 2, 'mba': 2.5, 
               'phd': 3, 'ph.d': 3, 'ba': 1, 'ma': 2,
               'postdoctoral': 4, 'postdoc': 4, 'doctorate': 3}


degree_dict2 = {'advanced degree': 2, 'ms or': 2, 'ms degree': 2, '4 year degree': 1, 'bs/': 1, 'ba/': 1,
                '4-year degree': 1, 'b.s.': 1, 'm.s.': 2, 'm.s': 2, 'b.s': 1, 'phd/': 3, 'ph.d.': 3, 'ms/': 2,
                'm.s/': 2, 'm.s./': 2, 'msc/': 2, 'master/': 2, 'master\'s/': 2, 'bachelor\s/': 1}
degree_keywords2 = set(degree_dict2.keys())

# Streamlining the Job Descriptions using NLP Techniques

In [10]:
df['job_summary'].iloc[5:]

5       Work on the development of the PropellerAds pr...
6       Good understanding of deep learning frameworks...
7       Run machine learning tests and experiments.,De...
8       Very good experience in Python/R, SQL or on re...
9       Minimum of 3+ years experience in developing s...
                              ...                        
1578    Experience with data visualization software su...
1579    Lead efforts to gather and analyze data requir...
1580    Linking millions of assured data points, Janes...
1581    Design and implement quantitative and qualitat...
1582    Analyze website data to determine areas of sit...
Name: job_summary, Length: 1578, dtype: object

In [11]:
word_tokenize(df['job_summary'].iloc[5])

['Work',
 'on',
 'the',
 'development',
 'of',
 'the',
 'PropellerAds',
 'product',
 ':',
 'conduct',
 'exploratory',
 'analysis',
 '-',
 'put',
 'forward',
 'hypotheses',
 ',',
 'select',
 'metrics',
 ',',
 'and',
 'conduct',
 'research',
 'and',
 'experiments…']

In [12]:
from nltk import pos_tag
from nltk.stem import PorterStemmer

pos_tag(tool_keywords1)

[('python', 'NN'),
 ('pytorch', 'NN'),
 ('sql', 'JJ'),
 ('mxnet', 'NN'),
 ('mlflow', 'NN'),
 ('einstein', 'NN'),
 ('theano', 'NN'),
 ('pyspark', 'NN'),
 ('solr', 'NN'),
 ('mahout', 'NN'),
 ('cassandra', 'NN'),
 ('aws', 'VBZ'),
 ('powerpoint', 'JJ'),
 ('spark', 'NN'),
 ('pig', 'NN'),
 ('sas', 'NN'),
 ('java', 'NN'),
 ('nosql', 'JJ'),
 ('docker', 'NN'),
 ('salesforce', 'NN'),
 ('scala', 'NN'),
 ('r', 'NN'),
 ('c', 'VBP'),
 ('c++', 'JJ'),
 ('net', 'JJ'),
 ('tableau', 'NN'),
 ('pandas', 'NN'),
 ('scikitlearn', 'NN'),
 ('sklearn', 'NN'),
 ('matlab', 'NN'),
 ('scala', 'NN'),
 ('keras', 'NNS'),
 ('tensorflow', 'JJ'),
 ('clojure', 'NN'),
 ('caffe', 'NN'),
 ('scipy', 'NN'),
 ('numpy', 'JJ'),
 ('matplotlib', 'NN'),
 ('vba', 'NN'),
 ('spss', 'NN'),
 ('linux', 'JJ'),
 ('azure', 'NN'),
 ('cloud', 'NN'),
 ('gcp', 'NN'),
 ('mongodb', 'NN'),
 ('mysql', 'NN'),
 ('oracle', 'NN'),
 ('redshift', 'NN'),
 ('snowflake', 'VBP'),
 ('kafka', 'NN'),
 ('javascript', 'NN'),
 ('qlik', 'NN'),
 ('jupyter', 'NN'),
 ('

# Final Processing of the Keywords and the Job Descriptions

In [13]:
from nltk import pos_tag
from nltk.stem import PorterStemmer

ps = PorterStemmer()


# process the job description.
def prepare_job_desc(desc):
    # tokenize description.
    tokens = word_tokenize(desc)
        
    # Parts of speech (POS) tag tokens.
    token_tag = pos_tag(tokens)
    
    # Only include some of the POS tags.
    include_tags = ['VBN', 'VBD', 'JJ', 'JJS', 'JJR', 'CD', 'NN', 'NNS', 'NNP', 'NNPS']
    filtered_tokens = [tok for tok, tag in token_tag if tag in include_tags]
    
    # stem words.
    stemmed_tokens = [ps.stem(tok).lower() for tok in filtered_tokens]
    return set(stemmed_tokens)

df['job_description_word_set'] = df['job_summary'].map(prepare_job_desc)

# process the keywords
tool_keywords1_set = set([ps.stem(tok) for tok in tool_keywords1]) # stem the keywords (since the job description is also stemmed.)
tool_keywords1_dict = {ps.stem(tok):tok for tok in tool_keywords1} # use this dictionary to revert the stemmed words back to the original.

skill_keywords1_set = set([ps.stem(tok) for tok in skill_keywords1])
skill_keywords1_dict = {ps.stem(tok):tok for tok in skill_keywords1}

degree_keywords1_set = set([ps.stem(tok) for tok in degree_dict.keys()])
degree_keywords1_dict = {ps.stem(tok):tok for tok in degree_dict.keys()}

In [14]:
df['job_description_word_set'].iloc[10]

{'algorithms.',
 'comput',
 'deep',
 'develop',
 'engin',
 'languag',
 'learn',
 'machin',
 'model',
 'natur',
 'nlp',
 'process',
 'python…',
 'vision'}

# Matching the Keywords and the Job Descriptions

In [15]:
tool_list = []
skill_list = []
degree_list = []

msk = df['company'] != '' # just in case you want to filter the data.

num_postings = len(df[msk].index)
for i in range(num_postings):
    job_desc = df[msk].iloc[i]['job_summary'].lower()
    job_desc_set = df[msk].iloc[i]['job_description_word_set']
    
    # check if the keywords are in the job description. Look for exact match by token.
    tool_words = tool_keywords1_set.intersection(job_desc_set)
    skill_words = skill_keywords1_set.intersection(job_desc_set)
    degree_words = degree_keywords1_set.intersection(job_desc_set)
    
    # check if longer keywords (more than one word) are in the job description. Match by substring.
    j = 0
    for tool_keyword2 in tool_keywords2:
        # tool keywords.
        if tool_keyword2 in job_desc:
            tool_list.append(tool_keyword2)
            j += 1
    
    k = 0
    for skill_keyword2 in skill_keywords2:
        # skill keywords.
        if skill_keyword2 in job_desc:
            skill_list.append(skill_keyword2)
            k += 1
            
       # search for the minimum education.
    min_education_level = 999
    for degree_word in degree_words:
        level = degree_dict[degree_keywords1_dict[degree_word]]
        min_education_level = min(min_education_level, level)
    
    for degree_keyword2 in degree_keywords2:
        # longer keywords. Match by substring.
        if degree_keyword2 in job_desc:
            level = degree_dict2[degree_keyword2]
            min_education_level = min(min_education_level, level)
    
    # label the job descriptions without any tool keywords.
    if len(tool_words) == 0 and j == 0:
        tool_list.append('nothing specified')
    
    # label the job descriptions without any skill keywords.
    if len(skill_words) == 0 and k == 0:
        skill_list.append('nothing specified')
    
    # If none of the keywords were found, but the word degree is present, then assume it's a bachelors level.
    if min_education_level > 500:
        if 'degree' in job_desc:
            min_education_level = 1
    
    tool_list += list(tool_words)
    skill_list += list(skill_words)
    degree_list.append(min_education_level)

# Visualizing the Results

# Top Tools In-Demand

In [16]:
# create the list of tools.
df_tool = pd.DataFrame(data={'cnt': tool_list})
df_tool = df_tool.replace(tool_keywords1_dict)

# group some of the categories together.
msk = df_tool['cnt'] == 'h20'
df_tool.loc[msk, 'cnt'] = 'h2o'

msk = df_tool['cnt'] == 'aws'
df_tool.loc[msk, 'cnt'] = 'amazon web services'

msk = df_tool['cnt'] == 'gcp'
df_tool.loc[msk, 'cnt'] = 'google cloud'

msk = df_tool['cnt'] == 'github'
df_tool.loc[msk, 'cnt'] = 'git'

msk = df_tool['cnt'] == 'postgressql'
df_tool.loc[msk, 'cnt'] = 'postgres'

msk = df_tool['cnt'] == 'tensor'
df_tool.loc[msk, 'cnt'] = 'tensorflow'

df_tool_top50 = df_tool['cnt'].value_counts().reset_index().rename(columns={'index': 'tool'}).iloc[:50]


In [17]:
# visualize the tools.
layout = dict(
    title='Tools For Data Scientists',
    yaxis=dict(
        title='% of job postings',
        tickformat=',.0%',
    )
)

fig = go.Figure(layout=layout)
fig.add_trace(go.Bar(
    x=df_tool_top50['tool'],
    y=df_tool_top50['cnt']/num_postings
))

iplot(fig)

# Top Skills In-Demand

In [18]:
# create the list of skills/knowledge.
df_skills = pd.DataFrame(data={'cnt': skill_list})
df_skills = df_skills.replace(skill_keywords1_dict)

# group some of the categories together.
msk = df_skills['cnt'] == 'nlp'
df_skills.loc[msk, 'cnt'] = 'natural language processing'

msk = df_skills['cnt'] == 'convolutional'
df_skills.loc[msk, 'cnt'] = 'convolutional neural network'

msk = df_skills['cnt'] == 'cnn'
df_skills.loc[msk, 'cnt'] = 'convolutional neural network'

msk = df_skills['cnt'] == 'recurrent'
df_skills.loc[msk, 'cnt'] = 'recurrent neural network'

msk = df_skills['cnt'] == 'rnn'
df_skills.loc[msk, 'cnt'] = 'recurrent neural network'

msk = df_skills['cnt'] == 'knn'
df_skills.loc[msk, 'cnt'] = 'nearest neighbors'

msk = df_skills['cnt'] == 'svm'
df_skills.loc[msk, 'cnt'] = 'support vector machine'

msk = df_skills['cnt'] == 'machine vision'
df_skills.loc[msk, 'cnt'] = 'computer vision'

msk = df_skills['cnt'] == 'ab testing'
df_skills.loc[msk, 'cnt'] = 'a/b testing'

df_skills_top50 = df_skills['cnt'].value_counts().reset_index().rename(columns={'index': 'skill'}).iloc[:50]


In [19]:
# visualize the skills.
layout = dict(
    title='Skills For Data Scientists',
    yaxis=dict(
        title='% of job postings',
        tickformat=',.0%',
    )
)

fig = go.Figure(layout=layout)
fig.add_trace(go.Bar(
    x=df_skills_top50['skill'],
    y=df_skills_top50['cnt']/num_postings
))

iplot(fig)

# Minimum Education Required

In [20]:
# create the list of degree.
df_degrees = pd.DataFrame(data={'cnt': degree_list})
df_degrees['degree_type'] = ''


msk = df_degrees['cnt'] == 1
df_degrees.loc[msk, 'degree_type'] = 'bachelors'

msk = df_degrees['cnt'] == 2
df_degrees.loc[msk, 'degree_type'] = 'masters'

msk = df_degrees['cnt'] == 3
df_degrees.loc[msk, 'degree_type'] = 'phd'

msk = df_degrees['cnt'] == 4
df_degrees.loc[msk, 'degree_type'] = 'postdoc'

msk = df_degrees['cnt'] == 2.5
df_degrees.loc[msk, 'degree_type'] = 'mba'

msk = df_degrees['cnt'] > 500
df_degrees.loc[msk, 'degree_type'] = 'not specified'


df_degree_cnt = df_degrees['degree_type'].value_counts().reset_index().rename(columns={'index': 'degree'}).iloc[:50]


In [21]:
# visualize the degrees.
layout = dict(
    title='Minimum Education For Data Scientists',
    yaxis=dict(
        title='% of job postings',
        tickformat=',.0%',
    )
)

fig = go.Figure(layout=layout)
fig.add_trace(go.Bar(
    x=df_degree_cnt['degree'],
    y=df_degree_cnt['degree_type']/num_postings
))

iplot(fig)